## How to edit an already existing Neuron Model Configuration

This notebook explains how to load an already existing neuron model configuration from nexus, edit it, and store it on nexus

To run this notebook, you need to already have a NeuronModelConfiguration in Nexus.

## Loading the NeuronModelConfiguration

Please fill in the values to match the metadata of the model you want to edit.

In [1]:
emodel = "L5_TPC"
etype = "cAC"
mtype = None
ttype = None
iteration_tag = "XXXX-XX-XX"
species = "mouse"
brain_region = "SSCX"

nexus_project = "" # specify the nexus project to use
nexus_organisation = "" # specify the nexus organisation to use
nexus_endpoint = "" # specify the nexus endpoint to use

# Advanced settings (only change if you know what you are doing)
forge_path = "./forge.yml"
forge_ontology_path = "./nsg.yml"

This next cell will ask for your Nexus token:

In [ ]:
from bluepyemodel.access_point.nexus import NexusAccessPoint

access_point = NexusAccessPoint(
    emodel=emodel,
    species=species,
    brain_region=brain_region,
    project=nexus_project,
    organisation=nexus_organisation,
    endpoint=nexus_endpoint,
    forge_path=forge_path,
    forge_ontology_path=forge_ontology_path,
    etype=etype,
    mtype=mtype,
    ttype=ttype,
    iteration_tag=iteration_tag
)

In [ ]:
from bluepyemodel.model.model_configurator import ModelConfigurator

configurator = ModelConfigurator(access_point)
configurator.load_configuration()

print(configurator.configuration)

## Adding Mechanisms, Distributions, Parameters or Morphology

You can now modify the neuron model configuration. Below is an example where a mechanism, a distribution, and a parameter are added to the existing mechanisms, distributions, and parameters of the neuron model configuration. Additionally, the morphology is updated to a different one.

In [4]:
from bluepyemodel.model.morphology_configuration import MorphologyConfiguration

In [5]:
# add distribution
configurator.configuration.add_distribution(
    distribution_name="decay",
    function="math.exp({distance}*{constant})*{value}",
    parameters=["constant"],
)

In [6]:
# add two parameters
configurator.configuration.add_parameter(
    parameter_name="constant",
    locations="distribution_decay",
    value=[-0.1,0.0],
    mechanism=None,
    distribution_name=None,
    stochastic=None,
)
configurator.configuration.add_parameter(
    parameter_name="gNaTgbar_NaTg",
    locations="apical",
    value=[0.0, 0.1],
    mechanism="NaTg",
    distribution_name="decay",
    stochastic=None,
)

In [7]:
# add mechanism
configurator.configuration.add_mechanism(
    mechanism_name="NaTg",
    locations="apical",
    stochastic=None,
    version=None,
)

In [8]:
# change morphology
configurator.configuration.morphology = MorphologyConfiguration(name="ak171123_C_idA")

## Removing Parameters, Mechanisms, and Distributions

Here, we will see how to remove parameters, mechanisms and distributions. For the sake of simplicity, we will remove the ones that we just added.

In [9]:
# remove parameter
configurator.configuration.remove_parameter(parameter_name="constant")

When a mechanism is removed, it also removes all associated parameters. Here, 'gNaTgbar_NaTg' parameter will be removed as well

In [10]:
# remove mechanism
configurator.configuration.remove_mechanism(mechanism_name="NaTg")

There is no function to remove a distribution, but if you need to remove one, you can use the code below.

In [11]:
# remove a distribution
distribution_name = "decay"
configurator.configuration.distributions = [d for d in configurator.configuration.distributions if d.name != distribution_name]

## Saving the NeuronModelConfiguration

If there is no optimised model in your access point, you can store the new neuron model configuration as it is.

However, if there is an optimised model in your access point, it is advisable to use a new access point with an unused iteration tag to avoid conflicts with the existing model.

In this example, since we have just created the access point without optimising any model yet, we will keep it for storing the new neuron model configuration. Note that the old NeuronModelConfiguration will be deprecated.

In [ ]:
# store new model config
configurator.save_configuration()

To verify that the optimization works, run the following commands:

In [ ]:
!python pipeline.py --step=extract --emodel={emodel} --etype={etype} --iteration_tag={iteration_tag} --ttype={ttype}
!python pipeline.py --step=test_optimise --emodel={emodel} --etype={etype} --iteration_tag={iteration_tag} --ttype={ttype}